In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [2]:
!pip install datasets

In [3]:
!pip install torch

In [4]:
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c

  Cloning https://github.com/huggingface/trl.git (to revision 7630f877f91c556d9e5a3baa4b6e2894d90ff84c) to /tmp/pip-req-build-dvam586v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-dvam586v
  Running command git rev-parse -q --verify 'sha^7630f877f91c556d9e5a3baa4b6e2894d90ff84c'
  Running command git fetch -q https://github.com/huggingface/trl.git 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Running command git checkout -q 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Resolved https://github.com/huggingface/trl.git to commit 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.7.12.dev0-py3-none-any.whl size=173433 sha256=eb512dbb821f8f192646d44eef9dbe9af3bc838281a2835bc739666019d8883b
  Stored in directory: /root/.cache/pip/wheels/ad/f5/b1/f5ac48230936583c88cfde8

# Importing the Modules

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import unicodedata
import string
from datasets import load_dataset
import re
import os
import io
from nltk import BigramCollocationFinder
import nltk.collocations


#Loading the Base Model

In [6]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-hf"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Loading the datasets and Preprocessing the datasets

In [7]:
import re
import os
import io
import nltk
from datasets import load_dataset
from nltk.collocations import BigramCollocationFinder

class AmharicPreprocessor:
    def __init__(self, expansion_file_dir='/content/llm-finetunning/short_forms.txt', bigram_dir='bigrams.txt'):
        self.expansion_file_dir = expansion_file_dir
        self.bigram_dir = bigram_dir
        self.short_form_dict = self.get_short_forms()

    def get_short_forms(self):
        exp = {}
        try:
            with open(self.expansion_file_dir, encoding='utf8') as text:
                for line in text:
                    line = line.strip()
                    if not line:
                        continue
                    expanded = line.split("-")
                    exp[expanded[0].strip()] = expanded[1].replace(" ", '_').strip()
        except FileNotFoundError:
            print(f"File not found: {self.expansion_file_dir}")
        return exp

    def expand_short_form(self, input_short_word):
        return self.short_form_dict.get(input_short_word, input_short_word)

    def normalize_char_level_mismatch(self, input_token):
        replacements = [
            ('[ሃኅኃሐሓኻ]', 'ሀ'), ('[ሑኁዅ]', 'ሁ'), ('[ኂሒኺ]', 'ሂ'), ('[ኌሔዄ]', 'ሄ'), ('[ሕኅ]', 'ህ'),
            ('[ኆሖኾ]', 'ሆ'), ('[ሠ]', 'ሰ'), ('[ሡ]', 'ሱ'), ('[ሢ]', 'ሲ'), ('[ሣ]', 'ሳ'), ('[ሤ]', 'ሴ'),
            ('[ሥ]', 'ስ'), ('[ሦ]', 'ሶ'), ('[ዓኣዐ]', 'አ'), ('[ዑ]', 'ኡ'), ('[ዒ]', 'ኢ'), ('[ዔ]', 'ኤ'),
            ('[ዕ]', 'እ'), ('[ዖ]', 'ኦ'), ('[ጸ]', 'ፀ'), ('[ጹ]', 'ፁ'), ('[ጺ]', 'ፂ'), ('[ጻ]', 'ፃ'),
            ('[ጼ]', 'ፄ'), ('[ጽ]', 'ፅ'), ('[ጾ]', 'ፆ'), ('(ሉ[ዋአ])', 'ሏ'), ('(ሙ[ዋአ])', 'ሟ'),
            ('(ቱ[ዋአ])', 'ቷ'), ('(ሩ[ዋአ])', 'ሯ'), ('(ሱ[ዋአ])', 'ሷ'), ('(ሹ[ዋአ])', 'ሿ'),
            ('(ቁ[ዋአ])', 'ቋ'), ('(ቡ[ዋአ])', 'ቧ'), ('(ቹ[ዋአ])', 'ቿ'), ('(ሁ[ዋአ])', 'ኋ'),
            ('(ኑ[ዋአ])', 'ኗ'), ('(ኙ[ዋአ])', 'ኟ'), ('(ኩ[ዋአ])', 'ኳ'), ('(ዙ[ዋአ])', 'ዟ'),
            ('(ጉ[ዋአ])', 'ጓ'), ('(ደ[ዋአ])', 'ዷ'), ('(ጡ[ዋአ])', 'ጧ'), ('(ጩ[ዋአ])', 'ጯ'),
            ('(ጹ[ዋአ])', 'ጿ'), ('(ፉ[ዋአ])', 'ፏ'), ('[ቊ]', 'ቁ'), ('[ኵ]', 'ኩ')
        ]
        for pattern, replacement in replacements:
            input_token = re.sub(pattern, replacement, str(input_token))
        return input_token

    def remove_punc_and_special_chars(self, text):
        return re.sub(r'[\!\@\#\$\%\^\«\»\&\*\(\)\…\[\]\{\}\;\“\”\›\’\‘\"\'\:\,\.\‹\/\<\>\?\\\\|\`\´\~\-\=\+\፡\።\፤\;\፦\፥\፧\፨\፠\፣]', '', text)

    def remove_ascii_and_numbers(self, text_input):
        rm_num_and_ascii = re.sub('[A-Za-z0-9]', '', text_input)
        return re.sub('[\u1369-\u137C]+', '', rm_num_and_ascii)


    def arabic2geez(self, arabicNumber):
        ETHIOPIC_ONE = 0x1369
        ETHIOPIC_TEN = 0x1372
        ETHIOPIC_HUNDRED = 0x137B
        ETHIOPIC_TEN_THOUSAND = 0x137C

        arabicNumber = str(arabicNumber)
        n = len(arabicNumber) - 1
        if n % 2 == 0:
            arabicNumber = "0" + arabicNumber
            n += 1

        arabicBigrams = [arabicNumber[i:i+2] for i in range(0, n, 2)]
        reversedArabic = arabicBigrams[::-1]
        geez = []

        for index, pair in enumerate(reversedArabic):
            curr_geez = ''
            artens = pair[0]
            arones = pair[1]
            amtens = ''
            amones = ''
            if artens != '0':
                amtens = str(chr((int(artens) + (ETHIOPIC_TEN - 1))))
            else:
                if arones == '0':
                    continue
            if arones != '0':
                amones = str(chr((int(arones) + (ETHIOPIC_ONE - 1))))
            if index > 0:
                if index % 2 != 0:
                    curr_geez = amtens + amones + str(chr(ETHIOPIC_HUNDRED))
                else:
                    curr_geez = amtens + amones + str(chr(ETHIOPIC_TEN_THOUSAND))
            else:
                curr_geez = amtens + amones
            geez.append(curr_geez)

        geez = ''.join(geez[::-1])
        if geez.startswith('፩፻') or geez.startswith('፩፼'):
            geez = geez[1:]

        if len(arabicNumber) >= 7:
            end_zeros = ''.join(re.findall('([0]+)$', arabicNumber)[0:])
            i = int(len(end_zeros) / 3)
            if len(end_zeros) >= (3 * i):
                if i >= 3:
                    i -= 1
                for thoushand in range(i - 1):
                    geez += '፼'

        return geez

    def get_expanded_number(self, number):
        if '.' not in str(number):
            return self.arabic2geez(number)
        else:
            num, decimal = str(number).split('.')
            if decimal.startswith('0'):
                decimal = decimal[1:]
                dot = ' ነጥብ ዜሮ '
            else:
                dot = ' ነጥብ '
            return self.arabic2geez(num) + dot + self.arabic2geez(decimal)

    def tokenize(self, corpus):
        sentences = re.compile('[!?።\፡\፡]+').split(corpus)
        tokens = []
        for sentence in sentences:
            tokens.extend(sentence.split())
        return tokens

    def collocation_finder(self, tokens):
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        finder = BigramCollocationFinder.from_words(tokens)
        finder.apply_freq_filter(3)
        frequent_bigrams = finder.nbest(bigram_measures.chi_sq, 5)

        with io.open(self.bigram_dir, "w", encoding="utf8") as PhraseWriter:
            for bigram in frequent_bigrams:
                PhraseWriter.write(bigram[0] + ' ' + bigram[1] + "\n")

    def normalize_multi_words(self, tokenized_sentence, corpus):
        bigram = set()
        sent_with_bigrams = []
        index = 0

        if not os.path.exists(self.bigram_dir):
            self.collocation_finder(self.tokenize(corpus))

        try:
            with open(self.bigram_dir, encoding='utf8') as phrase_file:
                for line in phrase_file:
                    bigram.add(tuple(line.strip().split()))
        except FileNotFoundError:
            print(f"File not found: {self.bigram_dir}")

        while index < len(tokenized_sentence):
            if index + 1 < len(tokenized_sentence) and (tokenized_sentence[index], tokenized_sentence[index + 1]) in bigram:
                sent_with_bigrams.append(
                    (tokenized_sentence[index] + "_" + tokenized_sentence[index + 1]))
                index += 1
            else:
                sent_with_bigrams.append(tokenized_sentence[index])
            index += 1
        return sent_with_bigrams

    def preprocess_text(self, text):
        text = self.expand_short_form(text)
        text = self.normalize_char_level_mismatch(text)
        text = self.remove_punc_and_special_chars(text)
        text = self.remove_ascii_and_numbers(text)
        return text


def load_huggingface_dataset(dataset_name):
    return load_dataset(dataset_name)

def preprocess_dataset(dataset, text_field, preprocessor):
    def preprocess_example(example):
        if text_field in example:
            example[text_field] = preprocessor.preprocess_text(example[text_field])
        return example

    return dataset.map(preprocess_example)

# List of dataset names
dataset_names = [
    "rasyosef/amharic-news-category-classification",
    "simonbutt/amharic_truthful_qa",
    "simonbutt/amharic_gsm8k",
    "EthioNLP/Amharic_LLAMA_MT",
    "EthioNLP/Amharic_Instruction_dataset",
    "Tvsybkzkmapab/Amharic_ad_generation",
    "BiniyamAjaw/amharic_dataset_v2",
    "Henok/amharic-qa"
]

# Load datasets and preprocess text fields
preprocessor = AmharicPreprocessor()
loaded_datasets = {}

for name in dataset_names:
    dataset = load_huggingface_dataset(name)
    print(f"Dataset: {name}")
    for split in dataset.keys():
        print(f"  Split: {split}")
        print(f"  Fields: {dataset[split].column_names}")

        # Identify the text field to preprocess
        text_fields = ['headline', 'article', 'question', 'am_question', 'am_answer', 'instruction', 'input', 'output', 'text', 'targets', 'inputs']
        for text_field in text_fields:
            if text_field in dataset[split].column_names:
                print(f"Preprocessing field: {text_field} in split: {split}")
                dataset[split] = preprocess_dataset(dataset[split], text_field, preprocessor)

    loaded_datasets[name] = dataset
    print()


File not found: /content/llm-finetunning/short_forms.txt


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/49971 [00:00<?, ? examples/s]

Dataset: rasyosef/amharic-news-category-classification
  Split: train
  Fields: ['headline', 'category', 'date', 'views', 'article', 'link', 'word_len', 'label']
Preprocessing field: headline in split: train


Map:   0%|          | 0/49971 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/263 [00:00<?, ? examples/s]

  Split: test
  Fields: ['targets', 'inputs']
Preprocessing field: targets in split: test


Map:   0%|          | 0/523 [00:00<?, ? examples/s]

Preprocessing field: inputs in split: test


Map:   0%|          | 0/523 [00:00<?, ? examples/s]

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

In [8]:
!pip install jupyter tornado tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00


In [9]:
!pip install tensorflow-gpu

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
from peft import LoraConfig

# Configuration parameters
model_name = "NousResearch/Llama-2-7b-hf"
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
output_dir = "./results"
num_train_epochs = 1
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

# Load the tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

llama_tokenizer = AutoTokenizer.from_pretrained(model_name)
llama_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=bnb_config,
)

# Set LoRA configuration
lora_config = LoraConfig(
    r=lora_r,
    alpha=lora_alpha,
    dropout=lora_dropout
)

# Set Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    max_grad_norm=max_grad_norm,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    optim=optim,
    lr_scheduler_type=lr_scheduler_type,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    save_steps=save_steps,
    logging_steps=logging_steps,
)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=llama_model,
    tokenizer=llama_tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]